🧩 1. Instalación de librerías necesarias

In [6]:
!pip install xarray netcdf4 pandas


📥 2. Carga de archivos .nc

In [7]:
import xarray as xr

# Abrimos ambos archivos descomprimidos
ds_temp = xr.open_dataset("datos_netcdf/data_stream-oper_stepType-instant.nc")
ds_acum = xr.open_dataset("datos_netcdf/data_stream-oper_stepType-accum.nc")


🧮 3. Procesamiento a DataFrame

In [8]:
import pandas as pd

df_temp = ds_temp[["t2m", "d2m"]].to_dataframe().reset_index()
df_acum = ds_acum[["tp", "ssrd"]].to_dataframe().reset_index()

df = pd.merge(df_temp, df_acum, on=["valid_time", "latitude", "longitude"])

df.rename(columns={
    "valid_time": "fecha",
    "t2m": "temperatura_2m_C",
    "d2m": "punto_rocio_2m_C",
    "tp": "precipitacion_total_mm",
    "ssrd": "radiacion_solar_whm2"
}, inplace=True)

# Conversión de unidades
df["temperatura_2m_C"] -= 273.15
df["punto_rocio_2m_C"] -= 273.15
df["precipitacion_total_mm"] *= 1000
df["radiacion_solar_whm2"] /= 3600

# Dejar solo fecha sin hora
df["fecha"] = pd.to_datetime(df["fecha"]).dt.date

# Agregación diaria
numericas = ["temperatura_2m_C", "punto_rocio_2m_C", "precipitacion_total_mm", "radiacion_solar_whm2"]
df_dia = df.groupby(["latitude", "longitude", "fecha"])[numericas].mean().reset_index()
df_dia.rename(columns={"latitude": "latitud", "longitude": "longitud"}, inplace=True)


🧮 4. Generación de características avanzadas

In [9]:
def add_moving_features(df, col, nombre_base):
    for dias in [4, 7, 14, 21, 28]:
        df[f"{nombre_base}_prom_{dias}d"] = (
            df.groupby(["latitud", "longitud"])[col]
            .transform(lambda x: x.rolling(window=dias, min_periods=1).mean())
        )
        df[f"{nombre_base}_acum_{dias}d"] = (
            df.groupby(["latitud", "longitud"])[col]
            .transform(lambda x: x.rolling(window=dias, min_periods=1).sum())
        )

add_moving_features(df_dia, "radiacion_solar_whm2", "radiacion_solar")
add_moving_features(df_dia, "temperatura_2m_C", "temperatura_2m")

for dias in [1, 2, 3, 4, 5]:
    df_dia[f"precipitacion_acum_{dias}d"] = (
        df_dia.groupby(["latitud", "longitud"])["precipitacion_total_mm"]
        .transform(lambda x: x.rolling(window=dias, min_periods=1).sum())
    )


🧮 5. Exportar a CSV

In [10]:
df_dia.to_csv("carbonspace_datos.csv", index=False)


| Variable                 | Descripción                                                      |
| ------------------------ | ---------------------------------------------------------------- |
| `latitud`                | Latitud del píxel o punto geográfico (°)                         |
| `longitud`               | Longitud del píxel o punto geográfico (°)                        |
| `fecha`                  | Fecha de observación (AAAA-MM-DD)                                |
| `temperatura_2m_C`       | Temperatura promedio diaria del aire a 2 metros (°C)             |
| `punto_rocio_2m_C`       | Temperatura promedio diaria del punto de rocío a 2 metros (°C)   |
| `precipitacion_total_mm` | Precipitación total diaria (mm)                                  |
| `radiacion_solar_whm2`   | Radiación solar superficial descendente acumulada diaria (Wh/m²) |

| Variables derivadas        | Descripción                                        |
| -------------------------- | -------------------------------------------------- |
| `radiacion_solar_prom_4d`  | Promedio de radiación solar en los últimos 4 días  |
| `radiacion_solar_acum_4d`  | Acumulado de radiación solar en los últimos 4 días |
| `radiacion_solar_prom_7d`  | Promedio de 7 días                                 |
| `radiacion_solar_acum_7d`  | Acumulado de 7 días                                |
| `radiacion_solar_prom_14d` | Promedio de 14 días                                |
| `radiacion_solar_acum_14d` | Acumulado de 14 días                               |
| `radiacion_solar_prom_21d` | Promedio de 21 días                                |
| `radiacion_solar_acum_21d` | Acumulado de 21 días                               |
| `radiacion_solar_prom_28d` | Promedio de 28 días                                |
| `radiacion_solar_acum_28d` | Acumulado de 28 días                               |
| `temperatura_2m_prom_4d`   | Promedio de temperatura de los últimos 4 días |
| `temperatura_2m_acum_4d`   | Suma de temperatura de los últimos 4 días     |
| `temperatura_2m_prom_7d`   | Promedio de 7 días                            |
| `temperatura_2m_acum_7d`   | Suma de 7 días                                |
| `temperatura_2m_prom_14d`  | Promedio de 14 días                           |
| `temperatura_2m_acum_14d`  | Suma de 14 días                               |
| `temperatura_2m_prom_21d`  | Promedio de 21 días                           |
| `temperatura_2m_acum_21d`  | Suma de 21 días                               |
| `temperatura_2m_prom_28d`  | Promedio de 28 días                           |
| `temperatura_2m_acum_28d`  | Suma de 28 días                               |
| `precipitacion_acum_1d`    | Precipitación acumulada de 1 día |
| `precipitacion_acum_2d`    | Acumulado de 2 días              |
| `precipitacion_acum_3d`    | Acumulado de 3 días              |
| `precipitacion_acum_4d`    | Acumulado de 4 días              |
| `precipitacion_acum_5d`    | Acumulado de 5 días              |
